In [ ]:
# Read jsonl file as a dict

import json
import pandas as pd

PATH = 'data/output_20231004.jsonl'

data = []
with open(PATH) as f:
    for line in f:
        data.append(json.loads(line))



In [ ]:
data[0]

In [ ]:
df = pd.DataFrame(
    data={
        '_annotator_id': [d['_annotator_id'] for d in data],
        '_timestamp': [d['_timestamp'] for d in data],
    }
)
df = df.groupby('_annotator_id').count()
df